In [1]:
users = ['Ryan', 'Danielle',  'Vijay', 'Chris']
movies = ['Star Wars', 'The Dark Knight', 'Shrek', 'The Incredibles', 'Bleu', 'Memento']
features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']

num_users = len(users)
num_movies = len(movies)
num_feats = len(features)
num_recommendations = 2

In [3]:
print(num_users, num_movies, num_feats)

4 6 5


In [2]:
import torch

# Users' ratings for movies
users_movies = torch.tensor([
    [4, 6, 8, 0, 0, 0],
    [0, 0, 10, 0, 8, 3],
    [0, 6, 0, 0, 3, 7],
    [10, 9, 0, 5, 0, 2]
], dtype=torch.float32)

# Features of the movies (one-hot encoded)
# Columns might represent categories like ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
movies_feats = torch.tensor([
    [1, 1, 0, 0, 1],
    [1, 1, 0, 0, 0],
    [0, 0, 1, 1, 0],
    [1, 0, 1, 1, 0],
    [0, 0, 0, 0, 1],
    [1, 0, 0, 0, 1]
], dtype=torch.float32)


In [4]:
# Compute users' feature vectors
users_feats = torch.matmul(users_movies, movies_feats)

In [5]:
users_feats

tensor([[10., 10.,  8.,  8.,  4.],
        [ 3.,  0., 10., 10., 11.],
        [13.,  6.,  0.,  0., 10.],
        [26., 19.,  5.,  5., 12.]])

In [6]:
# Normalize each user's feature vector to sum to 1
users_feats = users_feats / users_feats.sum(dim=1, keepdim=True)

In [8]:
print("Normalized User Feature Vectors:", users_feats)

Normalized User Feature Vectors: tensor([[0.2500, 0.2500, 0.2000, 0.2000, 0.1000],
        [0.0882, 0.0000, 0.2941, 0.2941, 0.3235],
        [0.4483, 0.2069, 0.0000, 0.0000, 0.3448],
        [0.3881, 0.2836, 0.0746, 0.0746, 0.1791]])


In [9]:
top_users_features = torch.topk(users_feats, k=num_feats, dim=1)[1]

In [10]:
top_users_features

tensor([[1, 0, 2, 3, 4],
        [4, 2, 3, 0, 1],
        [0, 4, 1, 3, 2],
        [0, 1, 4, 3, 2]])

In [11]:
for i in range(len(users)):
    selected_features = [features[index] for index in top_users_features[i]]
    print('{}: {}'.format(users[i], selected_features))

Ryan: ['Sci-Fi', 'Action', 'Comedy', 'Cartoon', 'Drama']
Danielle: ['Drama', 'Comedy', 'Cartoon', 'Action', 'Sci-Fi']
Vijay: ['Action', 'Drama', 'Sci-Fi', 'Cartoon', 'Comedy']
Chris: ['Action', 'Sci-Fi', 'Drama', 'Cartoon', 'Comedy']


In [12]:
users_ratings = torch.matmul(users_feats, movies_feats.t())
users_ratings

tensor([[0.6000, 0.5000, 0.4000, 0.6500, 0.1000, 0.3500],
        [0.4118, 0.0882, 0.5882, 0.6765, 0.3235, 0.4118],
        [1.0000, 0.6552, 0.0000, 0.4483, 0.3448, 0.7931],
        [0.8507, 0.6716, 0.1493, 0.5373, 0.1791, 0.5672]])

In [13]:
# we will mask the ratings matrix to set ratings to zero for movies that a user has already rated
users_movies_mask = users_movies > 0
users_ratings_new = torch.where(users_movies_mask, torch.zeros_like(users_ratings), users_ratings)

In [14]:
users_ratings_new

tensor([[0.0000, 0.0000, 0.0000, 0.6500, 0.1000, 0.3500],
        [0.4118, 0.0882, 0.0000, 0.6765, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000, 0.4483, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1493, 0.0000, 0.1791, 0.0000]])

In [15]:
top_movies = torch.topk(users_ratings_new, k=num_recommendations, dim=1)[1]

In [16]:
for i in range(len(users)):
    recommended_movie_names = [movies[index] for index in top_movies[i]]
    print('{}: {}'.format(users[i], recommended_movie_names))

Ryan: ['The Incredibles', 'Memento']
Danielle: ['The Incredibles', 'Star Wars']
Vijay: ['Star Wars', 'The Incredibles']
Chris: ['Bleu', 'Shrek']
